In [ ]:
import os
import json
import asyncio
from pydantic import BaseModel, Field
from functools import partial
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

# from dotenv import load_dotenv
from typing import List, Optional, Callable, Any

# load_dotenv()

api_key = os.getenv('OPEN_ROUTER_API_KEY')

if not api_key:
    raise ValueError("OPEN_ROUTER_API_KEY is missing.")

llm_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="google/gemini-2.5-flash-lite-preview-06-17",
    api_key=api_key,
    model_info={
        "family": 'gemini',
        "vision": True,
        "function_calling": True,
        "json_output": False,
        "structured_output": False
    }
)

class ParsedIdeaModel(BaseModel):
    problem: str
    solution: str
    domain: str

class MarketSizeModel(BaseModel):
    TAM: str
    SAM: str
    SOM: str

class SuggestedStack(BaseModel):
    frontend: str
    backend: str
    database: str
    infrastructure: str
    others: str

class TechFeasibilityModel(BaseModel):
    tech_feasibility: str
    tech_risks: Optional[List[str]] = None
    suggested_stack: Optional[SuggestedStack] = None

class Competitor(BaseModel):
    name: str
    description: str

class CompetitorModel(BaseModel):
    direct_competitors: Optional[List[Competitor]]
    indirect_competitors: Optional[List[Competitor]]
    differentiation_factors: Optional[List[str]]

class GTMStrategyModel(BaseModel):
    target_segments: Optional[List[str]]
    distribution_channels: Optional[List[str]]
    early_adopter_hooks: Optional[List[str]]
    initial_launch_plan: str
    growth_tactics: Optional[List[str]]

class RevenueModel(BaseModel):
    primary_revenue_model: str
    secondary_revenue_model: Optional[str]
    pricing_strategy: str
    value_capture_mechanism: str

class FinanceModel(BaseModel):
    revenue_streams: List[str]
    cost_structure: List[str]
    monthly_burn_rate_estimate: str
    twelve_month_revenue_projection: str
    unit_economics_insight: str

class AgentState(BaseModel):
    raw_idea: str
    parsed_idea_model: ParsedIdeaModel = None
    market_size_model: MarketSizeModel = None
    tech_feasibility_model: TechFeasibilityModel = None
    competitor_model: CompetitorModel = None
    gtm_strategy_model: GTMStrategyModel = None
    revenue_model: RevenueModel = None
    finance_model: FinanceModel = None

state = AgentState(raw_idea="Hi")


#### Prompts

In [19]:
agent_prompt_idea = """\
You are a startup idea parser agent.

Your task is to read a raw user-submitted startup idea — usually in free-form natural language — and extract the following three key structured fields:

1. **problem** - What issue, pain point, or inefficiency is the idea trying to solve?
2. **solution** - What is the proposed solution, technology, or service offered to address the problem?
3. **domain** - What broad industry, vertical, or category does this idea belong to (e.g., healthcare, education, fintech, logistics)?

Respond ONLY in JSON format using the following schema:

```json
{
  "problem": "<clearly defined problem statement>",
  "solution": "<clear explanation of the proposed solution>",
  "domain": "<relevant domain name>"
}"""

agent_prompt_market_size = """\
You are a market sizing agent that estimates TAM, SAM, and SOM for a given startup idea.

Your task is to analyze the provided startup idea description and return approximate, well-structured market size estimates for the following:

1. **TAM (Total Addressable Market)** - The total market demand for the product or service, assuming full global adoption across all segments.
2. **SAM (Serviceable Available Market)** - The portion of TAM targeted by the startup based on geography, vertical, or customer segment.
3. **SOM (Serviceable Obtainable Market)** - The realistic portion of SAM that the startup can capture in the next 3-5 years, considering current competition and resources.

Here is the required information to produce below result.
problem: {problem}
solution": {solution}
domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "TAM": "<global market size estimate with units, e.g., $120B>",
  "SAM": "<targeted market size estimate with justification>",
  "SOM": "<realistically obtainable market size with justification>"
}}
"""

agent_prompt_tech_feasibility = """\
You are a technical feasibility agent tasked with evaluating the technical implementation potential of a startup idea.

Your job is to analyze the provided startup concept and produce the following structured outputs:

1. **tech_feasibility** - A short paragraph (2-4 sentences) assessing how feasible the idea is from a technical perspective based on current tools, infrastructure, and maturity of required technologies.
2. **tech_risks** - A list of major technical challenges or uncertainties (e.g., scalability, data availability, algorithmic complexity, integration hurdles).
3. **suggested_stack** - A high-level proposed tech stack including backend, frontend, database, infrastructure, and any AI/ML or third-party service components where applicable.

Here is the startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "tech_feasibility": "<short narrative on feasibility>",
  "tech_risks": [
    "<risk 1>",
    "<risk 2>",
    "<risk 3>"
  ],
  "suggested_stack": {{
    "frontend": "<framework or library>",
    "backend": "<framework or language>",
    "database": "<db type>",
    "infrastructure": "<cloud/provider/services>",
    "others": "<any ML/AI/third-party tools if needed>"
  }}
}}
"""

agent_prompt_competitor = """\
You are a competitor landscape analysis agent tasked with identifying the startup's direct and indirect competitors, along with what differentiates it in the market.

Analyze the given startup idea and return:

1. **direct_competitors** - Companies or platforms offering the same or very similar services or products to the same target audience.
2. **indirect_competitors** - Companies solving a similar problem but using a different approach, or targeting a related audience or adjacent market.
3. **differentiation_factors** - 2-5 crisp, strategic reasons that explain how this startup differentiates itself from the competition.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "direct_competitors": [
    {{
      "name": "<Company Name>",
      "description": "<Short one-line summary of their offering>"
    }}
  ],
  "indirect_competitors": [
    {{
      "name": "<Company Name>",
      "description": "<Short one-line summary of their offering>"
    }}
  ],
  "differentiation_factors": [
    "<Key differentiator 1>",
    "<Key differentiator 2>",
    "<Key differentiator 3>"
  ]
}}
"""

agent_prompt_gtm_strategy = """\
You are a Go-To-Market (GTM) strategy agent for a startup.

Your role is to analyze a startup's idea and produce a structured GTM plan. Your focus is on identifying the most effective way to launch and grow the product in its target markets.

Analyze the startup context and return the following components:

1. **target_segments** - Who are the ideal first users or customers? Segment by geography, user type, institution type, or behavior.
2. **distribution_channels** – How will the product reach the users? Include digital, partnership-based, and physical channels if relevant.
3. **early_adopter_hooks** – What features, incentives, or value propositions will attract the first wave of users?
4. **initial_launch_plan** – Describe the first launch geography or market, with a brief rollout timeline or phase strategy.
5. **growth_tactics** – Tactics for scaling after launch (e.g., referral systems, campus ambassador programs, SEO, paid ads, partnerships).

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Respond strictly in the following JSON format:

```json
{{
  "target_segments": ["<segment 1>", "<segment 2>"],
  "distribution_channels": ["<channel 1>", "<channel 2>"],
  "early_adopter_hooks": ["<hook 1>", "<hook 2>"],
  "initial_launch_plan": "<concise 2–3 sentence rollout plan>",
  "growth_tactics": ["<growth tactic 1>", "<growth tactic 2>"]
}}
"""

agent_prompt_revenue_model = """\
You are a revenue model agent for an early-stage startup.

Your task is to analyze the startup's offering and propose a monetization strategy that fits the user base, product nature, and domain. Focus on real-world, simple revenue mechanisms used by similar companies or marketplaces.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}

Return the following fields in valid JSON format:

```json
{{
  "primary_revenue_model": "<Core way this business makes money (e.g., listing fees, subscriptions, commissions)>",
  "secondary_revenue_model": "<Optional secondary revenue stream (e.g., ads, data licensing, referral commissions)>",
  "pricing_strategy": "<One-line pricing plan: e.g., freemium, per-listing, per-month, tiered>",
  "value_capture_mechanism": "<How the platform captures value from transactions or usage>"
}}
"""

agent_prompt_finance = """\
You are a startup financial modeling agent.

Your task is to analyze a startup's business idea and provide an early-stage financial model focused on revenue, burn rate, and basic unit economics. Assume the company is pre-seed or seed stage, building toward product-market fit.

Startup context:
- Problem: {problem}
- Solution: {solution}
- Domain: {domain}
- Revenue model: {revenue_model}  (e.g., listing fees, subscriptions, ads)

Respond strictly in the following JSON format:

```json
{{
  "revenue_streams": [
    "<Primary revenue stream (e.g., landlord listing fees)>",
    "<Secondary revenue stream (e.g., student premium subscriptions)>"
  ],
  "cost_structure": [
    "<Main fixed or variable costs (e.g., engineering, cloud infra, marketing)>"
  ],
  "monthly_burn_rate_estimate": "<Approximate early-stage monthly burn (e.g., $25K-$50K)>",
  "twelve_month_revenue_projection": "<Expected revenue range or goal (e.g., $100K-$300K)>",
  "unit_economics_insight": "<Short comment on CAC vs LTV, payback period, or breakeven logic>"
}}
"""


#### Agents

In [20]:
idea_parser_agent = AssistantAgent(name="idea_parser_agent",
                        model_client=llm_client,
                        description="You are a helpful assistant to validate the StartUp idea.",
                        system_message=agent_prompt_idea)

market_size_agent = AssistantAgent(
    name="market_size_agent",
    description="Generates Go-To-Market strategy",
    model_client=llm_client,
    system_message=("You are a startup GTM expert. Return structured GTM strategy in JSON format."))

tech_feasibility_agent = AssistantAgent(
    name="tech_feasibility_agent",
    model_client=llm_client,
    description="Evaluates the technical feasibility and risk profile of a startup idea based on the current tech landscape.",
    system_message=(
        "You are a technical feasibility analyst for early-stage startups. "
        "Your job is to assess whether the proposed solution is realistically buildable, "
        "highlight key technical risks, and suggest a viable tech stack. "
        "You must return your output in JSON format with three fields: "
        "`tech_feasibility`, `tech_risks` (list), and `suggested_stack` (object)."
    )
)

competitor_agent = AssistantAgent(name="competitor_agent",
                       model_client=llm_client,
                       description="You are an helpful assistamt to perform the competitor analysis",
                       system_message=(
                        "You are a startup competitor analysis expert. "
                        "Your job is to evaluate a given startup idea and return a structured JSON output with: "
                        "`direct_competitors` (list of company name and one-line description), "
                        "`indirect_competitors` (list of adjacent companies), and "
                        "`differentiation_factors` (2-5 concise reasons explaining the startup's unique market position). "
                        "Always return a valid JSON structure matching the expected schema."
                    )
)

gtm_strategy_agent = AssistantAgent(name="gtm_strategy_agent",
                       model_client=llm_client,
                       description="go to market strategy agent",
                       system_message = """
You are a Go-To-Market (GTM) strategy agent for a startup.

Your role is to analyze a startup's idea and produce a structured GTM plan. Your focus is on identifying the most effective way to launch and grow the product in its target markets.

Analyze the startup context and return the following components:

1. target_segments – Who are the ideal first users or customers? Segment by geography, user type, institution type, or behavior.
2. distribution_channels – How will the product reach the users? Include digital, partnership-based, and physical channels if relevant.
3. early_adopter_hooks – What features, incentives, or value propositions will attract the first wave of users?
4. initial_launch_plan – Describe the first launch geography or market, with a brief rollout timeline or phase strategy.
5. growth_tactics – Tactics for scaling after launch (e.g., referral systems, campus ambassador programs, SEO, paid ads, partnerships).

Startup context:
- Problem: Companies struggle to monitor and maintain positive team morale, identify burnout risks, and track Diversity, Equity, and Inclusion (DEI) trends within remote and hybrid workforces due to the lack of direct observation and engagement.
- Solution: A Software-as-a-Service (SaaS) platform that leverages sentiment analysis on communication platforms like Slack and Microsoft Teams to provide actionable insights into employee morale, potential burnout, and DEI-related trends.
- Domain: HR Tech

Respond strictly in the following JSON format:

```json
{
  "target_segments": ["<segment 1>", "<segment 2>"],
  "distribution_channels": ["<channel 1>", "<channel 2>"],
  "early_adopter_hooks": ["<hook 1>", "<hook 2>"],
  "initial_launch_plan": "<concise 2–3 sentence rollout plan>",
  "growth_tactics": ["<growth tactic 1>", "<growth tactic 2>"]
}"""
)

revenue_agent = AssistantAgent(name="revenue_agent",
                       model_client=llm_client,
                       description="revenue agent",
                       system_message=(
                        "You are a revenue model strategist for early-stage startups. "
                        "Your job is to analyze the startup's offering and generate a structured, realistic monetization plan based on industry patterns. "
                        "Focus on revenue models that align with SaaS and B2B HR Tech solutions. "
                        "Return your output strictly in JSON format with the following fields: "
                        "`primary_revenue_model`, `secondary_revenue_model`, `pricing_strategy`, and `value_capture_mechanism`."
                    ))

finance_agent = AssistantAgent(name="finance_agent",
                       model_client=llm_client,
                       description="Finance agent",
                       system_message = (
                        "You are a financial modeling assistant for early-stage startups. "
                        "Your job is to generate a realistic, concise financial projection focusing on "
                        "revenue streams, cost structure, burn rate, and basic unit economics based on a provided startup context. "
                        "Assume the company is at the pre-seed or seed stage, aiming for product-market fit. "
                        "You must return your output in **valid JSON format** with the following fields:\n\n"
                        "`revenue_streams` (list of primary and secondary monetization sources),\n"
                        "`cost_structure` (list of key fixed/variable expenses),\n"
                        "`monthly_burn_rate_estimate` (string, e.g., \"$25K–$50K\"),\n"
                        "`twelve_month_revenue_projection` (string, e.g., \"$100K–$300K\"),\n"
                        "`unit_economics_insight` (short strategic commentary on LTV/CAC, payback period, or breakeven logic).\n\n"
                        "Ensure numbers are grounded in typical early-stage SaaS benchmarks. Use your best judgment to balance ambition with realism."
                    ))


#### Idea Parser

In [21]:
async def run_agent_idea_parser(state: AgentState, feedback: str="") -> ParsedIdeaModel:

    if not feedback:
        task = state.raw_idea
    else:
        task = state.raw_idea + f"\n\nNOTE: User feedback - {feedback}"

    result = await idea_parser_agent.run(task=task)

    idea_txt = result.messages[-1].content

    if idea_txt:

        if "```json" in idea_txt:
            idea_txt = idea_txt.replace("```json", "").replace("```", "").strip()

        idea_json = json.loads(idea_txt)
        parsed_model = ParsedIdeaModel(**idea_json)

        return parsed_model
    
    return ParsedIdeaModel(problem='', solution='', domain='')

#### Market

In [22]:
async def run_agent_marketsize(state: AgentState, feedback: str="") -> MarketSizeModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:
        
        prompt = agent_prompt_market_size.format(problem=state.parsed_idea_model.problem, 
                                                                solution=state.parsed_idea_model.solution, 
                                                                domain=state.parsed_idea_model.domain)
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"
            
        result = await market_size_agent.run(task=prompt)

        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            parsed_market_size = MarketSizeModel(**result_json)

            return parsed_market_size

    return MarketSizeModel(TAM='', SAM='', SOM='')

#### Tech Feasibility

In [23]:
async def run_agent_tech_feasibility(state: AgentState, feedback: str="") -> TechFeasibilityModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_tech_feasibility.format(
        problem=state.parsed_idea_model.problem,
        solution=state.parsed_idea_model.solution,
        domain=state.parsed_idea_model.domain)
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"
        
        result = await tech_feasibility_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            tech_feasibility_result = TechFeasibilityModel(**result_json)

            return tech_feasibility_result
        
    return TechFeasibilityModel(
    tech_feasibility="",
    tech_risks=[],
    suggested_stack=SuggestedStack(
        frontend="",
        backend="",
        database="",
        infrastructure="",
        others=""
    )
)

#### Competitor

In [24]:
async def run_agent_competitor(state: AgentState, feedback: str='') -> CompetitorModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_competitor.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await competitor_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            compitator_result = CompetitorModel(**result_json)

            return compitator_result
        
    return CompetitorModel(
        direct_competitors=[Competitor(name='', description='')],
        indirect_competitors=[Competitor(name='', description='')],
        differentiation_factors=[]
    )


#### GTM Strategy

In [25]:
async def run_agent_gtm_strategy(state: AgentState, feedback: str='') -> GTMStrategyModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_gtm_strategy.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await gtm_strategy_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            gtm_strategy_result = GTMStrategyModel(**result_json)

            return gtm_strategy_result
        
    return GTMStrategyModel(
    target_segments=[],
    distribution_channels=[],
    early_adopter_hooks=[],
    initial_launch_plan="",
    growth_tactics=[]
)

#### Revenue

In [26]:
async def run_agent_revenue(state: AgentState, feedback: str='') -> RevenueModel:

    if(
        state.parsed_idea_model.problem and 
        state.parsed_idea_model.solution and 
        state.parsed_idea_model.domain and 
        state.revenue_model
    ):
        prompt = agent_prompt_finance.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain,
            revenue_model=state.revenue_model.model_dump_json(indent=2)
        )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await revenue_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            revenue_result = RevenueModel(**result_json)

            return revenue_result
        
    return RevenueModel(primary_revenue_model='', secondary_revenue_model='', pricing_strategy='', value_capture_mechanism='')

#### Finance

In [27]:
async def run_agent_finance(state: AgentState, feedback: str='') -> FinanceModel:

    if state.parsed_idea_model.problem and state.parsed_idea_model.solution and state.parsed_idea_model.domain:

        prompt = agent_prompt_finance.format(
            problem=state.parsed_idea_model.problem,
            solution=state.parsed_idea_model.solution,
            domain=state.parsed_idea_model.domain,
            revenue_model=state.revenue_model.model_dump_json(indent=2)
            )
        
        if feedback:
            prompt = prompt + f"\n\nNOTE: User feedback - {feedback}"

        result = await finance_agent.run(task=prompt)
        result_txt = result.messages[-1].content

        if result_txt:
            
            if "```json" in result_txt:
                result_txt = result_txt.replace("```json", "").replace("```", "").strip()
            
            result_json = json.loads(result_txt)

            finance_result = FinanceModel(**result_json)

            return finance_result
        
    return FinanceModel(revenue_streams=[], cost_structure=[], monthly_burn_rate_estimate='', twelve_month_revenue_projection='', unit_economics_insight='')

#### Human in the loop

In [28]:
async def hitl_feedback_loop(output_name: str, initial_output: Any, rerun_fn: Callable[[str], Any]):
    attempt = 1
    max_attempt = 3
    current_output = initial_output

    while max_attempt >= attempt:

        print(f"\nOutput from {output_name} (Attempt #{attempt}):")
        print(json.dumps(current_output.model_dump(), indent=2))

        decision = input(f"\nApprove output from {output_name} (y/n)?").strip().lower()

        if decision == 'y':
            print(f"Approved: {current_output}")
            return current_output
        
        print("retrying...")
        feedback = input("Enter feedback to improve: ")
        current_output = await rerun_fn(feedback)

        attempt += 1

    print(f"max attempts reached for {output_name}, returning last output.")
    return current_output

#### Update Agent State

In [29]:
def update_agent_state(state: AgentState, **kwargs: Any) -> AgentState:
    """
    Returns a new AgentState with updated fields.
    
    Args:
        state: The current AgentState instance
        **kwargs: Fields to update in the state (e.g., parsed_idea_model=..., revenue_model=...)
    
    Returns:
        A new AgentState instance with updated fields
    """
    return state.model_copy(update=kwargs)

In [30]:
# startup_idea = input("Enter the startup idea: ").strip()

# if startup_idea:

#     state = update_agent_state(state, raw_idea=startup_idea)
#     result = await run_agent_idea_parser(state)
#     rerun_fn = partial(run_agent_idea_parser, state)
#     idea_result = await hitl_feedback_loop('Idea parser', result, rerun_fn)
#     state = update_agent_state(state, parsed_idea_model=idea_result)

#     result = await run_agent_marketsize(state)
#     rerun_fn = partial(run_agent_marketsize, state)
#     market_result = await hitl_feedback_loop('market research', result, rerun_fn)
#     state = update_agent_state(state, market_size_model=market_result)
    
#     result = await run_agent_tech_feasibility(state)
#     rerun_fn = partial(run_agent_tech_feasibility, state)
#     tech_feasibility_result = await hitl_feedback_loop('Tech feasibility', result, rerun_fn)
#     state = update_agent_state(state, tech_feasibility_model=tech_feasibility_result)
    
#     result = await run_agent_competitor(state)
#     rerun_fn = partial(run_agent_competitor, state)
#     compitator_result = await hitl_feedback_loop('Compitator', result, rerun_fn)
#     state = update_agent_state(state, competitor_model=compitator_result)

#     result = await run_agent_gtm_strategy(state)
#     rerun_fn = partial(run_agent_gtm_strategy, state)
#     gtm_strategy_result = await hitl_feedback_loop('GTM Strategy', result, rerun_fn)
#     state = update_agent_state(state, gtm_strategy_model=gtm_strategy_result)

#     result = await run_agent_revenue(state)
#     rerun_fn = partial(run_agent_revenue, state)
#     revenue_result = await hitl_feedback_loop('Revenue', result, rerun_fn)
#     state = update_agent_state(state, revenue_model=revenue_result)
    
#     result = await run_agent_finance(state)
#     rerun_fn = partial(run_agent_finance, state)
#     finance_result = await hitl_feedback_loop('Finance', result, rerun_fn)
#     state = update_agent_state(state, finance_model=finance_result)

#     print(f"Final:\n{state.model_dump_json(indent=2)}")



#### Orchestration

In [37]:

async def main():
    termination = TextMentionTermination(text='stop')

    roles = """
    market_size_agent : Estimates TAM, SAM, SOM based on the startup idea.
    competitor_agent : Identifies direct and indirect competitors and outlines differentiation strategies.
    finance_agent : Builds a basic early-stage financial model including revenue, cost, burn rate.
    revenue_agent : Determines monetization strategies like revenue models and pricing.
    tech_feasibility_agent : Analyzes technical feasibility and recommends a viable stack.
    gtm_strategy_agent : Designs go-to-market strategy including segments, channels, and launch plan.
    """

    participant_agents = [
                            market_size_agent,
                            tech_feasibility_agent,
                            competitor_agent,
                            gtm_strategy_agent,
                            revenue_agent,
                            finance_agent]
    
    selector_prompt = """
    You are a selector agent responsible for routing tasks to the correct specialized agent.

    Here are the available agents and their capabilities:
    {roles}

    You can choose from the following agents:
    {participants}

    Here is the conversation so far:
    {history}

    Based on the context, choose the most appropriate agent to handle the next task. 
    Return ONLY the agent name from the list above.
    """

    first_response = await idea_parser_agent.run(task=input("Enter startup idea for validation: "))

    team = SelectorGroupChat(
        participants=participant_agents,
        termination_condition=termination,
        max_turns=15,
        allow_repeated_speaker=True,
        model_client=llm_client,
        selector_prompt=selector_prompt
    )

    followup_task = "Continue startup evaluation using relevant agents."

    async for msg in team.run_stream(task="Continue evaluation using the idea: " + first_response.messages[-1].content):
        print(msg)


await main()

```json
{
  "problem": "Companies face challenges in objectively assessing and improving employee sentiment, detecting early signs of burnout, and monitoring trends related to Diversity, Equity, and Inclusion (DEI) within their remote and hybrid work environments.",
  "solution": "A Software-as-a-Service (SaaS) platform that leverages sentiment analysis on employee communications from platforms like Slack and Microsoft Teams to provide actionable insights into morale, burnout risk, and DEI metrics.",
  "domain": "HR Tech"
}
```
id='f35e70db-6c48-4e04-a37a-5ac9302c3583' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 17, 12, 8, 54, 158986, tzinfo=datetime.timezone.utc) content='Continue evaluation using the idea: ```json\n{\n  "problem": "Companies face challenges in objectively assessing and improving employee sentiment, detecting early signs of burnout, and monitoring trends related to Diversity, Equity, and Inclusion (DEI) within their remote and hyb

In [40]:
executed_agents = set()

def selector_func(history):
    global executed_agents
    agent_order = [
        "market_size_agent",
        "tech_feasibility_agent",
        "competitor_agent",
        "gtm_strategy_agent",
        "revenue_agent",
        "finance_agent"
    ]
    # Track which agents have already executed
    for msg in history:
        executed_agents.add(msg.source)

    if "revenue_agent" in executed_agents and "finance_agent" not in executed_agents:
        return "finance_agent"

    for agent in agent_order:
        if agent not in executed_agents:
            return agent

    return None  # fallback to model selection if all agents ran

participant_agents = [
                            market_size_agent,
                            tech_feasibility_agent,
                            competitor_agent,
                            gtm_strategy_agent,
                            revenue_agent,
                            finance_agent]

termination = TextMentionTermination(text='stop')

team = SelectorGroupChat(
    participants=participant_agents,
    termination_condition=termination,
    max_turns=10,
    allow_repeated_speaker=False,
    model_client=llm_client,
    selector_func=selector_func
)
raw_idea = input("Enter startup idea for validation: ")
first_response = await idea_parser_agent.run(task=raw_idea)

idea = first_response.messages[-1].content

followup_task = "Continue startup evaluation using relevant agents."

async for message in team.run_stream(task=followup_task + idea):
    print(message)


id='0676b195-c83e-4245-abb4-f13331623e16' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 17, 13, 3, 15, 989779, tzinfo=datetime.timezone.utc) content='Continue startup evaluation using relevant agents.```json\n{\n  "problem": "Organizations with remote and hybrid workforces find it difficult to objectively assess employee sentiment, identify risks of burnout, and monitor trends relating to Diversity, Equity, and Inclusion (DEI).",\n  "solution": "A SaaS platform that leverages sentiment analysis of employee communications on platforms like Slack and Microsoft Teams to track morale, burnout indicators, and DEI trends.",\n  "domain": "HR Tech"\n}\n```' type='TextMessage'
id='6ff0802a-b48d-46b5-98c5-35382ab9df45' source='competitor_agent' models_usage=RequestUsage(prompt_tokens=7428, completion_tokens=685) metadata={} created_at=datetime.datetime(2025, 7, 17, 13, 3, 21, 463664, tzinfo=datetime.timezone.utc) content='```json\n{\n  "go_to_market_strategy":

Error processing publish message for gtm_strategy_agent_dc7f01ec-7d0f-49f6-abf6-ba3df9ec78e3/dc7f01ec-7d0f-49f6-abf6-ba3df9ec78e3
Traceback (most recent call last):
  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-valida

RuntimeError: APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 65535 tokens, but can only afford 65454. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_2zc6pkuXw4NLCygL9miCoWS5hSg'}
Traceback:
Traceback (most recent call last):

  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 84, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 953, in on_messages_stream
    async for inference_output in self._call_llm(

  File "e:\Learning\Agentic_AI_V2_Projects\startup-idea-validator\venv\Lib\sit